In [2]:
pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 8.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 6.8 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 14.9 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827617 sha256=518277c2063266e416951725fe0a32785b2c81fe64e21ceb7553c20434523acf
  Stored in directory: /home/codespace/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.2 MB/s eta 0:00:00
  Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 18.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 14.6 MB/s eta 0:00:0000:010

In [5]:
pip install imageio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 7.5 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install ale-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install gym[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 11.4 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=cbdb861cfe02b649cdbaa63aa6d74e9240c1904134f419c6a4a427bcf3ffa109
  Stored in directory: /home/codespace/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all messages, 1 = filter out INFO, 2 = filter out WARNING, 3 = filter out ERROR
tf.get_logger().setLevel('ERROR')

import warnings
warnings.filterwarnings('ignore')


import logging
gym_logger = logging.getLogger('gym')
gym_logger.setLevel(logging.ERROR)

2023-12-09 12:08:57.488134: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-09 12:08:57.678489: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 12:08:57.678527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 12:08:57.707490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 12:08:57.745616: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-09 12:08:57.746603: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
import wandb
wandb.init(project="AC1", entity = "rl_proj")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/codespace/.netrc


In [4]:
import os
import gym
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import namedtuple, deque
from ale_py import ALEInterface
import wandb
import imageio

ale = ALEInterface() 

class Config:
    EPSILON_START = 1.0
    EPSILON_END = 0.01
    EPSILON_DECAY_RATE = 0.99
    EPISODES = 10  # Adjusted as per your request
    BATCH_SIZE = 128
    GAMMA = 0.999
    MAX_STEPS_PER_EPISODE = 1000
    LEARNING_RATE = 2.5e-4
    MEMORY_SIZE = 10000

config = Config()

env = gym.make("Assault-v4", render_mode="rgb_array")
n_actions = env.action_space.n

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
        self.transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

    def push(self, *args):
        self.memory.append(self.transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

class Actor(keras.Model):
    def __init__(self, n_actions):
        super(Actor, self).__init__()
        self.conv1 = layers.Conv2D(32, 8, strides=4, activation="relu")
        self.conv2 = layers.Conv2D(64, 4, strides=2, activation="relu")
        self.conv3 = layers.Conv2D(64, 3, strides=1, activation="relu")
        self.flatten = layers.Flatten()
        self.d1 = layers.Dense(512, activation="relu")
        self.d2 = layers.Dense(n_actions, activation="softmax")

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

class Critic(keras.Model):
    def __init__(self):
        super(Critic, self).__init__()
        self.conv1 = layers.Conv2D(32, 8, strides=4, activation="relu")
        self.conv2 = layers.Conv2D(64, 4, strides=2, activation="relu")
        self.conv3 = layers.Conv2D(64, 3, strides=1, activation="relu")
        self.flatten = layers.Flatten()
        self.d1 = layers.Dense(512, activation="relu")
        self.d2 = layers.Dense(1)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

actor_model = Actor(n_actions)
critic_model = Critic()

actor_optimizer = keras.optimizers.Adam(learning_rate=config.LEARNING_RATE)
critic_optimizer = keras.optimizers.Adam(learning_rate=config.LEARNING_RATE)

memory = ReplayMemory(config.MEMORY_SIZE)

def take_action(state, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        action_probabilities = actor_model.predict(state[np.newaxis, ...])
        return np.random.choice(n_actions, p=np.squeeze(action_probabilities))

def optimize_model():
    # Check if the memory has enough samples to form a batch
    if len(memory) < config.BATCH_SIZE:
        return  # Exit the function if not enough samples

    # Sample a batch of transitions from the replay memory
    transitions = memory.sample(config.BATCH_SIZE)
    # Transpose the batch (similar to zip(*transitions)) to separate states, actions, etc.
    batch = memory.transition(*zip(*transitions))

    # Convert the batches into numpy arrays for processing
    state_batch = np.array(batch.state)
    action_batch = np.array(batch.action)
    reward_batch = np.array(batch.reward)
    next_state_batch = np.array(batch.next_state)
    done_batch = np.array(batch.done)

    # Critic Update
    with tf.GradientTape() as tape:
        # Predict the current state's value
        values = critic_model(state_batch)
        # Predict the next state's value
        next_values = critic_model(next_state_batch)
        # Calculate the target value using the Bellman equation
        target_values = reward_batch + config.GAMMA * np.squeeze(next_values) * (1 - done_batch)
        # Calculate the loss as the mean squared error between target and predicted values
        critic_loss = tf.math.reduce_mean(tf.math.square(target_values - np.squeeze(values)))

    # Compute gradients of the critic loss with respect to the critic model's parameters
    critic_grads = tape.gradient(critic_loss, critic_model.trainable_variables)
    # Apply the gradients to the critic model's parameters
    critic_optimizer.apply_gradients(zip(critic_grads, critic_model.trainable_variables))

    # Actor Update
    with tf.GradientTape() as tape:
        # Predict the action probabilities for the current state
        action_probs = actor_model(state_batch)
        # Create a one-hot encoded mask for the taken actions
        action_mask = tf.one_hot(action_batch, n_actions)
        # Select the probabilities for the actions that were actually taken
        selected_action_probs = tf.reduce_sum(action_probs * action_mask, axis=1)

        # Calculate the advantage as the difference between target and predicted values
        advantage = target_values - np.squeeze(values)
        # Calculate the actor loss using the log probability of the actions times the advantage
        actor_loss = -tf.math.reduce_mean(tf.math.log(selected_action_probs) * advantage)

    # Compute gradients of the actor loss with respect to the actor model's parameters
    actor_grads = tape.gradient(actor_loss, actor_model.trainable_variables)
    # Apply the gradients to the actor model's parameters
    actor_optimizer.apply_gradients(zip(actor_grads, actor_model.trainable_variables))

epsilon = config.EPSILON_START
for episode in range(config.EPISODES):
    state, info = env.reset()
    state = state / 255.0
    done = False
    episode_reward = 0
    steps = 0

    frames = []

    while not done and steps < config.MAX_STEPS_PER_EPISODE:
        action = take_action(state, epsilon)
        next_state, reward, done, _, info = env.step(action)
        next_state = next_state / 255.0

        memory.push(state, action, next_state, reward, done)
        optimize_model()

        frames.append(env.render(mode="rgb_array"))

        state = next_state
        episode_reward += reward
        steps += 1

    epsilon = max((epsilon * config.EPSILON_DECAY_RATE), config.EPSILON_END)
    print(f"Episode: {episode+1}, Reward: {episode_reward}, Epsilon: {epsilon}")

    gif_path = f"episode_{episode+1}.gif"
    imageio.mimsave(gif_path, frames, fps=30)

    # Log episode metrics and GIF to wandb
    wandb.log({"episode": episode + 1, "reward": episode_reward, "epsilon": epsilon, "episode_gif": wandb.Video(gif_path, fps=4, format="gif")})

env.close()


TypeError: AtariEnv.render() got an unexpected keyword argument 'mode'